In [1]:
pip install metagpt==0.5.2

In [2]:
pip install aiocron discord

In [3]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup

async def fetch_html(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()

async def parse_main_page(html):
    title_list = []
    href_list = []
    soup = BeautifulSoup(html, 'html.parser')
    # 更新查找标签的逻辑以匹配当前网页结构
    title_tags = soup.find_all('h3', class_='mb-1 text-lg font-semibold leading-[1.2] hover:underline peer-hover:underline md:text-2xl')
    for title_tag in title_tags:
        a_tag = title_tag.find('a')  # 标题内的<a>标签
        if a_tag:
            title = a_tag.text.strip()  # 清除空白字符得到标题文本
            href = a_tag['href']  # 提取href属性
            title_list.append(title)  # 添加标题到列表
            href_list.append(href)  # 添加链接到列表
    return title_list, href_list

async def parse_sub_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    abstract = soup.find('div', class_="pb-8 pr-4 md:pr-16").p.text
    arxiv_url = soup.find('a', class_="btn inline-flex h-9 items-center", href=True)['href']
    return abstract, arxiv_url

async def main():
    url = 'https://huggingface.co/papers'
    base_url = 'https://huggingface.co'
    repositories = []
    try:
        html = await fetch_html(url)
        title_list, href_list = await parse_main_page(html)

        for title, href in zip(title_list, href_list):
            repo_info = {}
            repo_info['title'] = title
            # repo_info['href'] = href
            repositories.append(repo_info)
            # print(title, href)
            sub_html = await fetch_html(base_url + href)
            abstract, arxiv_url = await parse_sub_page(sub_html)
            # print(abstract, arxiv_url)
            repo_info['abstract'] = abstract
            repo_info['arxiv_url'] = arxiv_url
            repositories.append(repo_info)
        return repositories
    except Exception as e:
        print(f"An error occurred: {e}")




In [4]:
repositories = await main()
repositories

[{'title': 'InstantID: Zero-shot Identity-Preserving Generation in Seconds',
  'abstract': 'There has been significant progress in personalized image synthesis with\nmethods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world\napplicability is hindered by high storage demands, lengthy fine-tuning\nprocesses, and the need for multiple reference images. Conversely, existing ID\nembedding-based methods, while requiring only a single forward inference, face\nchallenges: they either necessitate extensive fine-tuning across numerous model\nparameters, lack compatibility with community pre-trained models, or fail to\nmaintain high face fidelity. Addressing these limitations, we introduce\nInstantID, a powerful diffusion model-based solution. Our plug-and-play module\nadeptly handles image personalization in various styles using just a single\nfacial image, while ensuring high fidelity. To achieve this, we design a novel\nIdentityNet by imposing strong semantic and weak spat

## Action: CrawlHuggingfaceDailyPaper

In [5]:
from metagpt.actions.action import Action
from metagpt.config import CONFIG

class CrawlHuggingfaceDailyPaper(Action):
    """
    This class specifically targets the daily papers section of the Huggingface website.
    Its main functionality includes asynchronously fetching and parsing the latest research papers
    published on Huggingface, extracting relevant details such as titles, abstracts, and arXiv URLs.
    It can be utilized in applications where up-to-date research information from Huggingface
    is required, making it a valuable tool for researchers and developers in AI and machine learning.
    """

    async def run(self, url: str = "https://huggingface.co/papers"):
        async with aiohttp.ClientSession() as client:
            async with client.get(url, proxy=CONFIG.global_proxy) as response:
                response.raise_for_status()
                html = await response.text()

        title_list, href_list = await parse_main_page(html)

        repositories = []
        base_url = 'https://huggingface.co'

        for title, href in zip(title_list, href_list):
            repo_info = {'title': title}
            sub_html = await fetch_html(base_url + href)
            abstract, arxiv_url = await parse_sub_page(sub_html)
            repo_info['abstract'] = abstract
            repo_info['arxiv_url'] = arxiv_url

            repositories.append(repo_info)

        return repositories


2024-01-17 06:09:21.841 | INFO     | metagpt.const:get_metagpt_package_root:32 - Package root set to /content


In [6]:
craw_paper_action = CrawlHuggingfaceDailyPaper()
resp = await craw_paper_action.run()
resp

[{'title': 'InstantID: Zero-shot Identity-Preserving Generation in Seconds',
  'abstract': 'There has been significant progress in personalized image synthesis with\nmethods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world\napplicability is hindered by high storage demands, lengthy fine-tuning\nprocesses, and the need for multiple reference images. Conversely, existing ID\nembedding-based methods, while requiring only a single forward inference, face\nchallenges: they either necessitate extensive fine-tuning across numerous model\nparameters, lack compatibility with community pre-trained models, or fail to\nmaintain high face fidelity. Addressing these limitations, we introduce\nInstantID, a powerful diffusion model-based solution. Our plug-and-play module\nadeptly handles image personalization in various styles using just a single\nfacial image, while ensuring high fidelity. To achieve this, we design a novel\nIdentityNet by imposing strong semantic and weak spat

### Action: SummaryDailyPaper

summary each daily paper, add five keywords by LLM

In [7]:
from typing import Any
PAPER_SUMMARY_PROMPT = """
    Transform the given data about a research paper into a neat Markdown format. Also, identify and include five relevant keywords that best represent the core themes of the paper.
    The provided data is:
    ```
    {data}
    ```
    Please create a markdown summary and suggest five keywords related to this paper.
    """
class SummaryDailyPaper(Action):

    async def run(
        self,
        data: Any
    ):
        return await self._aask(PAPER_SUMMARY_PROMPT.format(data=data))

In [8]:
await SummaryDailyPaper().run(resp[0])

# InstantID: Zero-shot Identity-Preserving Generation in Seconds

**Abstract:** There has been significant progress in personalized image synthesis with methods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world applicability is hindered by high storage demands, lengthy fine-tuning processes, and the need for multiple reference images. Conversely, existing ID embedding-based methods, while requiring only a single forward inference, face challenges: they either necessitate extensive fine-tuning across numerous model parameters, lack compatibility with community pre-trained models, or fail to maintain high face fidelity. Addressing these limitations, we introduce InstantID, a powerful diffusion model-based solution. Our plug-and-play module adeptly handles image personalization in various styles using just a single facial image, while ensuring high fidelity. To achieve this, we design a novel IdentityNet by imposing strong semantic and weak spatial conditions, integra

'# InstantID: Zero-shot Identity-Preserving Generation in Seconds\n\n**Abstract:** There has been significant progress in personalized image synthesis with methods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world applicability is hindered by high storage demands, lengthy fine-tuning processes, and the need for multiple reference images. Conversely, existing ID embedding-based methods, while requiring only a single forward inference, face challenges: they either necessitate extensive fine-tuning across numerous model parameters, lack compatibility with community pre-trained models, or fail to maintain high face fidelity. Addressing these limitations, we introduce InstantID, a powerful diffusion model-based solution. Our plug-and-play module adeptly handles image personalization in various styles using just a single facial image, while ensuring high fidelity. To achieve this, we design a novel IdentityNet by imposing strong semantic and weak spatial conditions, inte

## Role: DailyPaperWatcher

for analyze huggingfacce daily papers, and summary.

In [20]:
from typing import Dict, List
from metagpt.utils.common import OutputParser
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

class DailyPaperWatcher(Role):
    def __init__(
        self,
        name="Huggy",
        profile="DailyPaperWatcher",
        goal="Generate a summary of Huggingface daily papers.",
        constraints="Only analyze based on the provided Huggingface daily papers.",
    ):
        super().__init__(name, profile, goal, constraints)
        self._init_actions([CrawlHuggingfaceDailyPaper])

    # async def _act(self) -> Message:
    #     logger.info(f"{self._setting}: ready to {self._rc.todo}")

    #     todo = self._rc.todo

    #     # Retrieve the last message from memory
    #     try:
    #         msg = self.get_memories(k=1)[0]
    #     except IndexError:
    #         logger.error("No messages in memory")
    #         return Message(content="Error: No messages in memory", role=self.profile)

    #     # Perform the action
    #     try:
    #         result = await todo.run(msg.content)
    #         msg = Message(content=str(result), role=self.profile, cause_by=type(todo))
    #         self._rc.memory.add(msg)

    #         if isinstance(todo, (CrawlHuggingfaceDailyPaper, SummaryDailyPaper)):
    #             await self._handle_paper(result)
    #     except Exception as e:
    #         logger.error(f"Error during action execution: {e}")
    #         return Message(content=f"Error: {e}", role=self.profile)

    #     return result
    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self._rc.todo}")

        todo = self._rc.todo

        try:
            msg = self.get_memories(k=1)[0]
        except IndexError:
            logger.error("No messages in memory")
            return Message(content="Error: No messages in memory", role=self.profile)

        try:
            result = await todo.run(msg.content)
            if isinstance(todo, CrawlHuggingfaceDailyPaper):
                # 针对每篇论文创建并执行 SummaryDailyPaper 动作
                logger.info(f"Preparing to summarize {len(result)} papers")
                msg_content = ''
                for paper in result:
                    summary_action = SummaryDailyPaper(paper)
                    summary_result = await summary_action.run(paper)
                    summary_msg = Message(content=str(summary_result), role=self.profile, cause_by=type(summary_action))
                    self._rc.memory.add(summary_msg)
                    msg_content += str(summary_result)
                    msg_content += '\n'

            else:
                msg = Message(content=str(result), role=self.profile, cause_by=type(todo))
                self._rc.memory.add(msg)

        except Exception as e:
            logger.error(f"Error during action execution: {e}")
            return Message(content=f"Error: {e}", role=self.profile)

        return Message(content=str(msg_content), role=self.profile, cause_by=type(todo))

    async def _think(self) -> None:
        """Determine the next action to be taken by the role."""
        if self._rc.todo is None:
            self._set_state(0)
            return

        if self._rc.state + 1 < len(self._states):
            self._set_state(self._rc.state + 1)
        else:
            self._rc.todo = None

    async def _react(self) -> Message:
        """Execute the assistant's think and actions."""
        while True:
            await self._think()
            if self._rc.todo is None:
                break
            msg = await self._act()

        return msg

    async def _handle_paper(self, paper_info) -> None:
        actions = []
        # Enhanced logging for debugging
        logger.debug(f"Handling paper with info: {paper_info}")

        for paper in paper_info:
            actions.append(SummaryDailyPaper(paper))
            logger.info(f"Preparing to summarize paper: {paper['title']}")

        self._init_actions(actions)
        self._rc.todo = None



In [21]:
async def main():

    role = DailyPaperWatcher()
    result = await role.run("https://huggingface.co/papers")
    logger.info(result)


await main()

2024-01-17 06:15:47.538 | INFO     | __main__:_act:44 - Huggy(DailyPaperWatcher): ready to CrawlHuggingfaceDailyPaper
2024-01-17 06:15:48.725 | INFO     | __main__:_act:58 - Preparing to summarize 4 papers


# InstantID: Zero-shot Identity-Preserving Generation in Seconds

**Abstract:** There has been significant progress in personalized image synthesis with methods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world applicability is hindered by high storage demands, lengthy fine-tuning processes, and the need for multiple reference images. Conversely, existing ID embedding-based methods, while requiring only a single forward inference, face challenges: they either necessitate extensive fine-tuning across numerous model parameters, lack compatibility with community pre-trained models, or fail to maintain high face fidelity. Addressing these limitations, we introduce InstantID, a powerful diffusion model-based solution. Our plug-and-play module adeptly handles image personalization in various styles using just a single facial image, while ensuring high fidelity. To achieve this, we design a novel IdentityNet by imposing strong semantic and weak spatial conditions, integra

2024-01-17 06:16:17.635 | INFO     | __main__:main:5 - DailyPaperWatcher: # InstantID: Zero-shot Identity-Preserving Generation in Seconds

**Abstract:** There has been significant progress in personalized image synthesis with methods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world applicability is hindered by high storage demands, lengthy fine-tuning processes, and the need for multiple reference images. Conversely, existing ID embedding-based methods, while requiring only a single forward inference, face challenges: they either necessitate extensive fine-tuning across numerous model parameters, lack compatibility with community pre-trained models, or fail to maintain high face fidelity. Addressing these limitations, we introduce InstantID, a powerful diffusion model-based solution. Our plug-and-play module adeptly handles image personalization in various styles using just a single facial image, while ensuring high fidelity. To achieve this, we design a novel Id

 generalization to new objects or compositions.

**Keywords:** generative modeling, 3D assets, text-to-3D generation, diffusion models, HexaGen3D


## Trigger

In [11]:
import time
from aiocron import crontab
from typing import Optional
from pytz import BaseTzInfo
from pydantic import BaseModel, Field
from metagpt.schema import Message

class DailyPaperInfo(BaseModel):
    url: str
    timestamp: float = Field(default_factory=time.time)



class HuggingfaceDailyPaperCronTrigger():

    def __init__(self, spec: str, tz: Optional[BaseTzInfo] = None, url: str = "https://huggingface.co/papers") -> None:
        self.crontab = crontab(spec, tz=tz)
        self.url = url

    def __aiter__(self):
        return self

    async def __anext__(self):
        await self.crontab.next()
        return Message(self.url, DailyPaperInfo(url=self.url))


## Callback

In [12]:
### Discord
from google.colab import userdata

TOKEN = userdata.get('DISCORD_TOKEN')
CHANNEL_ID = userdata.get('DISCORD_CHANNEL_ID')

In [13]:
# callback
import os
import discord
async def discord_callback(msg: Message):
    intents = discord.Intents.default()
    intents.message_content = True
    intents.members = True

    client = discord.Client(intents=intents)
    token = TOKEN
    channel_id = int(CHANNEL_ID)

    async with client:
        await client.login(token)
        channel = await client.fetch_channel(channel_id)
        lines = []
        for i in msg.content.splitlines():
            if i.startswith(("# ", "## ", "### ")):
                if lines:
                    await channel.send("\n".join(lines))
                    lines = []
            lines.append(i)

        if lines:
            await channel.send("\n".join(lines))

## Main

In [24]:
from metagpt.subscription import SubscriptionRunner
# 运行入口，
async def main(spec: str = "54 16 * * *", discord: bool = True, wxpusher: bool = False):
    callbacks = []
    if discord:
        callbacks.append(discord_callback)

    if wxpusher:
        callbacks.append(wxpusher_callback)

    if not callbacks:
        async def _print(msg: Message):
            print(msg.content)
        callbacks.append(_print)

    async def callback(msg):
        await asyncio.gather(*(call(msg) for call in callbacks))

    runner = SubscriptionRunner()
    await runner.subscribe(DailyPaperWatcher(), HuggingfaceDailyPaperCronTrigger(spec), callback)
    await runner.run()

In [25]:
from pytz import timezone
from datetime import datetime, timedelta

current_time = datetime.now()
target_time = current_time + timedelta(minutes=1)
cron_expression = target_time.strftime('%M %H %d %m %w')
print(cron_expression)

17 06 17 01 3


In [26]:
await main(cron_expression)

2024-01-17 06:17:00.002 | INFO     | __main__:_act:44 - Huggy(DailyPaperWatcher): ready to CrawlHuggingfaceDailyPaper
2024-01-17 06:17:01.171 | INFO     | __main__:_act:58 - Preparing to summarize 4 papers


# InstantID: Zero-shot Identity-Preserving Generation in Seconds

**Abstract:** There has been significant progress in personalized image synthesis with methods such as Textual Inversion, DreamBooth, and LoRA. Yet, their real-world applicability is hindered by high storage demands, lengthy fine-tuning processes, and the need for multiple reference images. Conversely, existing ID embedding-based methods, while requiring only a single forward inference, face challenges: they either necessitate extensive fine-tuning across numerous model parameters, lack compatibility with community pre-trained models, or fail to maintain high face fidelity. Addressing these limitations, we introduce InstantID, a powerful diffusion model-based solution. Our plug-and-play module adeptly handles image personalization in various styles using just a single facial image, while ensuring high fidelity. To achieve this, we design a novel IdentityNet by imposing strong semantic and weak spatial conditions, integra

CancelledError: 